In [ ]:
import sys
import os.path
import pikepdf
import urllib.request
import pandas as pd
from datetime import datetime
from pathlib import Path
#import requests
Path("temp").mkdir(parents=True, exist_ok=True)
Path("data").mkdir(parents=True, exist_ok=True)

In [ ]:
os.remove("data/meta.csv")

In [ ]:
#loop
int=1
with open("data/PDFURLs.csv") as urls:
    for pdf_url in urls:

        
        unique = pdf_url.removeprefix("https://bildungsportal.sachsen.de/opal/oer/")
        unique = unique.removesuffix("\n")
        metadict = {"Identifier":unique}
        pfad = "temp/" + unique + ".pdf"
        if (not os.path.exists(pfad)):
            print("downloading:", unique)
            try:
                pdf_filename, headers = urllib.request.urlretrieve(pdf_url, pfad)
            except urllib.error.HTTPError as e:
                print("Error while downloading.")
                if e.code == 404:
                    metadict.update({"Error":'Missing(404)'})
                else:
                    metadict.update({"Error":'Unknown Downloaderror'})
                frame = pd.DataFrame(metadict, [0])
                if (os.path.exists("data/meta.csv")):   
                    oldframe = pd.read_csv("data/meta.csv")
                    newframe = pd.concat([oldframe, frame])
                    newframe.to_csv("data/meta.csv", index=False)           
                else:
                    frame.to_csv("data/meta.csv", index=False)
                metadict.clear
                int=int+1
                continue
        else:
            pdf_filename = pfad
            
        try:
            pdf = pikepdf.Pdf.open(pdf_filename)           
        except pikepdf._qpdf.PasswordError:
            print("This pdf requires a password.")
            metadict.update({"Error":'PasswordProtected'})
            frame = pd.DataFrame(metadict, [0])
            if (os.path.exists("data/meta.csv")):   
                oldframe = pd.read_csv("data/meta.csv")
                newframe = pd.concat([oldframe, frame])
                newframe.to_csv("data/meta.csv", index=False)           
            else:
                frame.to_csv("data/meta.csv", index=False)
            metadict.clear
            int=int+1
            continue
        except:
            print("An Error occured while opening.")
            metadict.update({"Error":'OpeningError'})
            frame = pd.DataFrame(metadict, [0])
            if (os.path.exists("data/meta.csv")):   
                oldframe = pd.read_csv("data/meta.csv")
                newframe = pd.concat([oldframe, frame])
                newframe.to_csv("data/meta.csv", index=False)           
            else:
                frame.to_csv("data/meta.csv", index=False)
            metadict.clear
            int=int+1
            continue
            
        docinfo = pdf.docinfo
        metadict = {"Identifier":unique}



        try:
            docinfo["/Author"]
        except KeyError:
            metadict.update({"Author":""})
        else:
            metadict.update({"Author":str(docinfo["/Author"])})

        
        try:
            docinfo["/CreationDate"]
        except KeyError:
            metadict.update({"Datum der Erstellung":""})
        else:
            creation = str(docinfo["/CreationDate"])
            try:
                date = datetime.strptime(creation.replace("'", ""), "D:%Y%m%d%H%M%S%z")
            except ValueError:
                metadict.update({"Datum der Erstellung":""})
            else:
                metadict.update({"Datum der Erstellung":date.strftime("%x")}) #TODO: change to normal dates
    
        try:
            docinfo["/Title"]
        except KeyError:
            metadict.update({"Titel":""})
        else:
            metadict.update({"Titel":str(docinfo["/Title"])})
    
        metadict.update({"Seitenzahl":len(pdf.pages)})

        if (pdf.pages[0].mediabox[2] - pdf.pages[0].mediabox[0]) > (pdf.pages[0].mediabox[3] - pdf.pages[0].mediabox[1]):
            metadict.update({"Format":"Dokument"})
        else:
            metadict.update({"Format":"Präsentation"})
    
        size = os.stat(pdf_filename).st_size
        if (size > 1000000000):
            size = size / 1000000000
            sizestr =str(round(size, 1)) + " GB"
        elif (size > 1000000):                    
            size = size / 1000000
            sizestr =str(round(size, 1)) + " MB"
        elif (size > 1000):
            size = size / 1000
            sizestr =str(round(size, 1)) + " kB"   
        else:
            sizestr =str(round(size, 1)) + " B"
        metadict.update({"Dateigröße":str(sizestr)})

        frame = pd.DataFrame(metadict, [0])
        if (os.path.exists("data/meta.csv")):   
            oldframe = pd.read_csv("data/meta.csv")
            newframe = pd.concat([oldframe, frame])
            newframe.to_csv("data/meta.csv", index=False)           
        else:
            frame.to_csv("data/meta.csv", index=False)
        metadict.clear
        pdf.close()
        print("finshed:", int)
        int=int+1